# YCP Hacks 2024
***Team: Sanij, Priyanshu, Andry, Marko (SPAM)***

This is the codespace where we trained a transfer learning model for predicting different skin conditions using the **Dermnet** dataset (https://www.kaggle.com/datasets/shubhamgoel27/dermnet/code).



In [1]:
# importing the libraries (do not remove or add libraries)
from typing import List, Set, Dict, Tuple, Optional
from PIL import Image
import os
import pathlib
import cv2
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shubhamgoel27/dermnet")

print("Path to dataset files:", path)

100%|██████████| 1.72G/1.72G [00:59<00:00, 31.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/shubhamgoel27/dermnet/versions/1


# Loading data

In [3]:
data_path = path+'/train'

train_data = []
val_data = []

for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    file = os.listdir(folder_path)
    num_train = int(0.8 * len(file))
    files_train = random.sample(file, num_train)
    files_val = list(set(file) - set(files_train))

    for file in files_train:
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224,224))
        train_data.append((img, folder))

    for file in files_val:
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224,224))
        val_data.append((img, folder))

# Set up model and Train

In [4]:
# prompt: set up the imported data and train it on a MobileNetV2 transfer learning model

# Define the image size and batch size
IMG_WIDTH = 224
IMG_HEIGHT = 224
BATCH_SIZE = 32

base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False
num_classes = 23

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

transfer_model = Model(inputs=base_model.input, outputs=predictions)
transfer_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#preprocess data
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

#train_data = [(preprocess_input(input), label) for input, label in train_data]
#val_data = [(preprocess_input(input), label) for input, label in val_data]

X_train, y_train = zip(*train_data)
X_val, y_val = zip(*val_data)

X_train = preprocess_input(np.array(X_train))
X_val = preprocess_input(np.array(X_val))

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

y_train_one_hot = to_categorical(y_train_encoded, num_classes)
y_val_one_hot = to_categorical(y_val_encoded, num_classes)


# Train the model
from keras.callbacks import ModelCheckpoint, EarlyStopping
custom_early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=0.001,
    mode='min'
)

EPOCHS = 20
BATCH_SIZE = 64
history = transfer_model.fit(X_train, y_train_one_hot, validation_data=(X_val, y_val_one_hot),
                   epochs = EPOCHS, batch_size=BATCH_SIZE,callbacks=[custom_early_stopping])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 41s 146ms/step - accuracy: 0.2423 - loss: 2.6679 - val_accuracy: 0.3206 - val_loss: 2.3172
Epoch 2/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 13s 68ms/step - accuracy: 0.3833 - loss: 2.0446 - val_accuracy: 0.3549 - val_loss: 2.2301
Epoch 3/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 13s 69ms/step - accuracy: 0.4776 - loss: 1.7471 - val_accuracy: 0.3713 - val_loss: 2.1747
Epoch 4/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 13s 68ms/step - accuracy: 0.5279 - loss: 1.5462 - val_accuracy: 0.3937 - val_loss: 2.1049
Epoch 5/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 13s 68ms/step - accuracy: 0.6096 - loss: 1.2938 - val_accuracy: 0.4075 - val_loss: 2.1153
Epoch 6/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 13s 69ms/step - accuracy: 0.6782 - loss: 1.0822 - val_accuracy: 0.4113 - val_loss: 2.1787
Epoch 7/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 13s 69ms/step - accuracy: 0.7549 - loss: 0.8614 - val_accuracy: 0.4178 - val_loss: 2.2174
Epoch 8/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 

In [5]:
transfer_model.save('tf_model.keras')